In [137]:
import cv2 as cv
import numpy as np
import os
from windowcapture import WindowCapture

# import win32gui, win32ui, win32con
import easyocr
import pandas as pd
from pywinauto import Desktop
import warnings
import pyautogui
import win32clipboard
import win32gui

warnings.filterwarnings("ignore", category=UserWarning)

# Initialize easyocr
reader = easyocr.Reader(lang_list=["en"])

# Initialize WindowCapture class with get_netplay_name function
# netplay_name = WindowCapture.get_netplay_name()

wincap = WindowCapture(window_name="NetPlay")
screenshot = wincap.get_screenshot()


In [138]:
screenshot.shape

(573, 723, 3)

In [139]:
player_location = pyautogui.locate(
    needleImage=r"C:\Users\Cooper\cv-mk64\img\netplay\Player.png",
    haystackImage=screenshot,
)

player_center = pyautogui.center(player_location)

mapping_location = pyautogui.locate(
    needleImage=r"C:\Users\Cooper\cv-mk64\img\netplay\Mapping.png",
    haystackImage=screenshot,
)

mapping_center = pyautogui.center(mapping_location)

print(f"Player Center: {player_center}")
print(f"Mapping Center: {mapping_center}")

player_screen_center = wincap.get_screen_position(player_center)
mapping_screen_center = wincap.get_screen_position(mapping_center)

print(f"Player Center Relative to Window Location on Screen: {player_screen_center}")
print(f"Mapping Center Relative to Window Location on Screen: {mapping_screen_center}")


Player Center: Point(x=383, y=133)
Mapping Center: Point(x=555, y=132)
Player Center Relative to Window Location on Screen: (1310, 490)
Mapping Center Relative to Window Location on Screen: (1482, 489)


In [140]:
pyautogui.FAILSAFE = False

In [99]:
player_screen_center[0]

-1366

In [112]:
def copy_paste():
    pyautogui.hotkey("ctrl", "c")
    win32clipboard.OpenClipboard()
    data = win32clipboard.GetClipboardData()
    win32clipboard.EmptyClipboard()
    win32clipboard.CloseClipboard()

    return data

In [130]:
# Bring the window to the front with the Window Handle ID
win32gui.SetForegroundWindow(wincap.hwnd)

pyautogui.click(player_screen_center[0], player_screen_center[1])
pyautogui.move(0, 25)
pyautogui.click()
player_one = copy_paste()
print(player_one)


pyautogui.click(mapping_screen_center[0], mapping_screen_center[1])
pyautogui.move(0, 25)
pyautogui.click()
mapping_one = copy_paste()
print(mapping_one)

pyautogui.click(player_screen_center[0], player_screen_center[1])
pyautogui.move(0, 50)
pyautogui.click()
player_two = copy_paste()
print(player_two)

pyautogui.click(mapping_screen_center[0], mapping_screen_center[1])
pyautogui.move(0, 50)
pyautogui.click()
mapping_two = copy_paste()
print(mapping_two)

COOPER
GC1,2
Blake
None


In [142]:
def create_player_frame(
    player_location: tuple[int, int],
    mapping_location: tuple[int, int],
    wincap: WindowCapture,
) -> pd.DataFrame:

    # Translate Player Location Tuple into X and Y variables
    player_x = player_location[0]
    player_y = player_location[1]

    # Translate Mapping Location Tuple into X and Y variables
    mapping_x = mapping_location[0]
    mapping_y = mapping_location[1]

    # Bring the window to the front with the Window Handle ID
    win32gui.SetForegroundWindow(wincap.hwnd)

    # Abstraction for pyautogui interaction
    # Helper Function to copy selected cell value and paste into variable
    def click_helper(x: int, y: int, y_move: int) -> None:
        pyautogui.click(x, y)
        pyautogui.move(0, y_move)
        pyautogui.click()
        pyautogui.hotkey("ctrl", "c")
        
    def copy_paste() -> str:
        win32clipboard.OpenClipboard()
        data = win32clipboard.GetClipboardData()
        win32clipboard.EmptyClipboard()
        win32clipboard.CloseClipboard()
        
        return data

    # Copy Player Data from NetPlay Window
    click_helper(x=player_x, y=player_y, y_move=25)
    player_one = copy_paste()

    click_helper(x=player_x, y=player_y, y_move=50)
    player_two = copy_paste()

    click_helper(x=player_x, y=player_y, y_move=75)
    player_three = copy_paste()

    click_helper(x=player_x, y=player_y, y_move=100)
    player_four = copy_paste()

    # Copy Controller Mapping (Port) Data from NetPlay Window
    click_helper(x=mapping_x, y=mapping_y, y_move=25)
    mapping_one = copy_paste()

    click_helper(x=mapping_x, y=mapping_y, y_move=50)
    mapping_two = copy_paste()

    click_helper(x=mapping_x, y=mapping_y, y_move=75)
    mapping_three = copy_paste()

    click_helper(x=mapping_x, y=mapping_y, y_move=100)
    mapping_four = copy_paste()

    df = pd.DataFrame(
        data={
            "Player": [player_one, player_two, player_three, player_four],
            "Port": [mapping_one, mapping_two, mapping_three, mapping_four],
        }
    )

    return df


In [147]:
df = create_player_frame(
    player_location=player_screen_center,
    mapping_location=mapping_screen_center,
    wincap=wincap,
)

In [148]:
df

,Player,Port
0,COOPER,"GC1,2"
1,Blake,None
2,Blake,None
3,Blake,Blake


In [ ]:
pyautogui.move(0, 110)

In [ ]:

pyautogui.click(button_center.x, button_center.y, button="left")
pyautogui.move(0, 110)
pyautogui.click()